# Visual Odometry pipeline
A naive visual odometry implementation with OpenCV & Python

In [ ]:
# Fetch data
!bash ./fetch_data.sh

### Run with KITTI
Make sure to download the KITTI odometry dataset and set KITTI=True

### Run with ad-hoc video
Set KITTI variable to False and input variable to the video

In [ ]:
import pdb
import io
import cv2
import time
import numpy as np
import PIL.Image
from matplotlib import pyplot as plt
from IPython import display
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform
from os import listdir
from os.path import isfile, join

seq = None
mono_folder = None
data_folder = None
train_image_names = None
train_labels = None

KITTI = True
input = './videos/test_countryroad.mp4'
# input = 0

if KITTI == True:
    seq = '00'
    mono_folder = 'image_2'
    data_root = '/Users/haydenpennington/dev/data/kitti/'
    data_folder = data_root + 'sequences/' + seq + '/' + mono_folder + '/'
    train_image_names = sorted([f for f in listdir(data_folder) if isfile(join(data_folder, f))])
    
    with open(data_root + 'poses/' + seq + '.txt', 'r') as f:
        train_labels = [x.split() for x in f.readlines()]
        train_labels = np.array([[float(x) for x in y] for y in train_labels])
        train_labels = train_labels.reshape(-1, 3, 4)
        

# print(train_image_names)

## Create the ORB feature detector
orb = cv2.ORB_create()


In [ ]:
def normalize(Kinv, pts):
    pts = np.concatenate([pts, np.array([1.0])])
    return np.dot(Kinv, pts).T[:2]

In [ ]:
def poseRt(R, t):
  t = t.reshape((3,))
  ret = np.eye(4)
  ret[:3, :3] = R
  ret[:3, 3] = t
  return ret

def fundamentalToRt(R, t):
  return np.linalg.inv(poseRt(R, t))

In [ ]:
def match_frames(frame1, frame2, corners1, corners2, kps1, kps2, descriptors1, descriptors2):
    bf_matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf_matcher.knnMatch(descriptors1, descriptors2, k=2)
    matches = np.asarray(matches)
    
    pairs = []
    norm_pairs = []
    indices = []

    # W,H = 1920, 1080
    # W, H = 1280, 720
    H, W = frame1.shape
    # W,H = 1920 // 4, 1080 // 4
    F = W // 2
    K = np.array([[F,0,W//2],[0,F,H//2],[0,0,1]])
    Kinv = np.linalg.inv(K)

    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            pt1 = np.asarray(kps1[m.queryIdx].pt)
            pt2 = np.asarray(kps2[m.trainIdx].pt)
            pairs.append((pt1, pt2))
            pt1 = normalize(Kinv, pt1)
            pt2 = normalize(Kinv, pt2)
            norm_pairs.append((pt1, pt2))
            indices.append((m.queryIdx, m.trainIdx))

    pairs = np.asarray(pairs)
    norm_pairs = np.asarray(norm_pairs)

    if pairs[:, 0].shape[0] >= 12 and pairs[:, 1].shape[0] >= 12:

        try:
            model, inliers = ransac(
                (norm_pairs[:, 0], norm_pairs[:, 1]),
                FundamentalMatrixTransform,
                min_samples=12,
                residual_threshold=0.001,
                max_trials=250
            )

            # return norm_pairs, pairs, K, indices
            return norm_pairs[inliers], pairs[inliers], model.params, K, indices
        except ValueError as error:
            print(error)
            return None
        
    else:
        return None


In [ ]:
def display_mat(M, fmt='jpeg'):
    f = io.BytesIO()
    PIL.Image.fromarray(M).save(f, fmt)
    display.display(display.Image(data=f.getvalue()))

In [ ]:
def triangulate(proj1, proj2, pts1, pts2):
    points4d = cv2.triangulatePoints(proj1, proj2, pts1, pts2).reshape(-1, 4)
    points3d = cv2.convertPointsFromHomogeneous(points4d)
    return points3d

# def triangulate_geohot(pose1, pose2, pts1, pts2):
#   ret = np.zeros((pts1.shape[0], 4))
#   for i, p in enumerate(zip(pts1, pts2)):
#     A = np.zeros((4,4))
#     A[0] = p[0][0] * pose1[2] - pose1[0]
#     A[1] = p[0][1] * pose1[2] - pose1[1]
#     A[2] = p[1][0] * pose2[2] - pose2[0]
#     A[3] = p[1][1] * pose2[2] - pose2[1]
#     _, _, vt = np.linalg.svd(A)
#     ret[i] = vt[3]
#     ret[i, :3] /= vt[3, 3]
#   return ret[:, :3]

def triangulate_geohot(pose1, pose2, pts1, pts2):
  ret = np.zeros((pts1.shape[0], 4))
  for i, p in enumerate(zip(pts1, pts2)):
    A = np.zeros((4,4))
    A[0] = p[0][0] * pose1[2] - pose1[0]
    A[1] = p[0][1] * pose1[2] - pose1[1]
    A[2] = p[1][0] * pose2[2] - pose2[0]
    A[3] = p[1][1] * pose2[2] - pose2[1]
    _, _, vt = np.linalg.svd(A)
    ret[i] = vt[3]
  return ret

In [ ]:
def pixel2cam(X, W, H):
    X = np.array(X).reshape(-1, 2)
    X[:, 0] /= W
    X[:, 1] /= H
    return X

In [ ]:
cap = None
if KITTI == False:
    cap = cv2.VideoCapture(input)
    # cap = cv2.VideoCapture(0)

last = None
last_corners = None
# last_keypoints = None
last_kps = None
last_descriptors = None

R_abs = np.eye(3)
t_abs = np.zeros(3)
last_R = np.eye(3)
last_t = np.zeros(3)
t_abs_all = []
t_abs_gt = []
last_norm_pairs = None
point_cloud = None
last_proj = None

i = 0
%matplotlib widget
while (KITTI == True and i < len(train_image_names)) or (cap is not None and cap.isOpened()):
    t0 = time.time()
    ret, im_original, label = None, None, None
    if KITTI == True:
        ret, im_original, label = True, cv2.imread(data_folder + train_image_names[i]), train_labels[i]
    else:
        ret, im_original = cap.read()

    if ret:
        im = im_original
        im = cv2.resize(im_original, (640, 480))
        multiplier_x = im_original.shape[1] / im.shape[1]
        multiplier_y = im_original.shape[0] / im.shape[0]
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        H, W = im.shape
        # print('(H, W)', H, W)
        
        if last is not None:
            im1 = im
            im2 = last

            corners = cv2.goodFeaturesToTrack(
                im1,
                3000,
                0.001,
                6
            )
            
            key_points1 = [cv2.KeyPoint(x=x[0][0], y=x[0][1], size=20) for x in corners]
            kps, descriptors = orb.compute(im1, key_points1)

            result = match_frames(im1, im2, corners, last_corners, kps, last_kps, descriptors, last_descriptors)
            if result is not None:
                norm_pairs, feature_pairs, F, K, indices = result
                # norm_pairs, feature_pairs, K, indices = result

                try:
                    # E, _ = cv2.findEssentialMat(norm_pairs[:, 0], norm_pairs[:, 1], 525, (640, 480), cv2.RANSAC)
                    # F, _ = cv2.findFundamentalMat(norm_pairs[:, 0], norm_pairs[:, 1], cv2.FM_LMEDS)
                    _, R, t, _ = cv2.recoverPose(F, norm_pairs[:, 0], norm_pairs[:, 1])
                    t = t.flatten()
                
                    R_abs = np.dot(R, R_abs)
                    scale = 0.8
                    t_abs = t_abs + R_abs.dot((t * scale))

                    # print(t_abs)
                    t_abs_all.append(t_abs)
                    if KITTI == True:
                        t_abs_gt.append(label[:3, 3])

                    KK = np.eye(4)
                    KK[:3, :3] = K
                    
                    Rt = np.eye(4)
                    # Rt[:3, :3] = R_abs
                    # Rt[:3, 3] = t_abs.T
                    # proj1 = np.dot(KK, Rt)
                    # proj1 = np.dot(KK, Rt)
                    
                    # proj1 = np.eye(4)
                    proj1 = Rt

                    if last_proj is not None:
                        Rt = np.eye(4)
                        Rt[:3, :3] = R
                        Rt[:3, 3] = t.T
                        RT = fundamentalToRt(R, t)
                        proj1 = np.dot(RT, last_proj)
                        # print(proj1)
                        # print(last_proj)
                        # pdb.set_trace()
                        # proj1 = KK.dot(Rt)
                        # proj1 = np.dot(np.dot(Rt, KK), last_proj)
                        k1 = []
                        k2 = []
                        
                        if last_norm_pairs is not None:
                            for index_pair in indices:
                                a, b = index_pair
                                Kinv = np.linalg.inv(K)
                                k1.append(normalize(Kinv, kps[a].pt))
                                k2.append(normalize(Kinv, last_kps[b].pt))
                            
                            kps1 = np.array(k1)
                            kps2 = np.array(k2)
                            # kps1 = kps1.reshape(2, -1)
                            # kps2 = kps2.reshape(2, -1)

                            # assert kps1[0, 0] == feature_pairs[0, 0, 0] / W
                            # assert kps1[0, 1] == feature_pairs[0, 0, 1] / H
                            
                            # points3d = t_abs + triangulate(proj1, last_proj, kps1, kps2).dot(R_abs)
                            # points3d = triangulate_geohot(proj1, last_proj, kps1, kps2)[:, :3]

                            # proj1 = np.eye(4)
                            # last_proj = np.eye(4)
                            # last_proj[:3, 3] = np.array([7, 8, 9]).T

                            # kps1 = np.array([10, 20, 30, 40]).reshape(-1, 2)
                            # kps2 = np.array([50, 60, 70, 80]).reshape(-1, 2)
                            # print(proj1)
                        
                            points4d = triangulate_geohot(proj1[:3, :4], last_proj[:3, :4], kps1, kps2)
                            # print(points4d.shape)
                            points3d = (points4d / points4d[:, 3:])[:, :3]
                            # pts3d = []
                            # for point in points3d:
                            #     if point[2] > 0:
                            #         pts3d.append(point)
                            # points3d = np.array(pts3d)
                            # pdb.set_trace()

                            if point_cloud is None:
                                point_cloud = points3d.reshape(-1, 3)
                            else:
                                point_cloud = np.concatenate([point_cloud, points3d.reshape(-1, 3)]).reshape(-1, 3)
                    
                    last_proj = proj1
                            
                    last_norm_pairs = norm_pairs

                    
                    for pt1, pt2 in feature_pairs:
                        u1,v1 = int(round(pt1[0]) * multiplier_x), int(round(pt1[1]) * multiplier_y)
                        u2,v2 = int(round(pt2[0]) * multiplier_x), int(round(pt2[1]) * multiplier_y)
                        cv2.circle(im_original, (u1, v1), color=(0, 255, 0), radius=3)
                        cv2.line(im_original, (u1, v1), (u2, v2), color=(255, 0, 0))
                        cv2.circle(im_original, (u2, v2), color=(0, 0, 255), radius=3)

                except ValueError as error:
                    print('error', error)

        last = im
        if last_corners is None:
            last_corners = cv2.goodFeaturesToTrack(
                im,
                3000,
                0.001,
                9
            )
            
            keypoints = [cv2.KeyPoint(x=x[0][0], y=x[0][1], size=20) for x in last_corners]
            last_kps, last_descriptors = orb.compute(im, keypoints)
        else:
            last_corners = corners
            last_kps, last_descriptors = kps, descriptors

        cv2.imshow('im1', im_original)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

        if (i > 100 and i % 1000 == 0) or (i > 0 and i < 11 and i % 10 == 0):
            display_mat(cv2.cvtColor(im_original, cv2.COLOR_BGR2RGB))

        t1 = time.time()
        # print('fps: ', 1 / (t1 - t0))


        i += 1

    else:
        break

cv2.destroyAllWindows()
cv2.waitKey(4)
if KITTI == False:
    cap.release()

## Red vs Green
Red is the ground truth pose and green is the predicted pose of the camera.

In [ ]:
# demo_crop_data = o3d.data.DemoCropPointCloud()
# pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
# vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
# chair = vol.crop_point_cloud(pcd)
# o3d.visualization.draw_geometries([chair],
#                                   zoom=0.7,
#                                   front=[0.5439, -0.2333, -0.8060],
#                                   lookat=[2.4615, 2.1331, 1.338],
#                                   up=[-0.1781, -0.9708, 0.1608])

In [ ]:
# import open3d as o3d

# pcd = o3d.geometry.PointCloud()
# # point_cloud[point_cloud < 0] = -10000
# pcd.points = o3d.utility.Vector3dVector(point_cloud)
# # Flip it, otherwise the pointcloud will be upside down.
# pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# # vol = o3d.visualization.SelectionPolygonVolume()

# # vol.bounding_polygon = o3d.utility.Vector3dVector(pcd.get_axis_aligned_bounding_box())
# # chair = o3d.geometry.crop_point_cloud(pcd, pcd.get_min_bound(), pcd.get_max_bound())
# # geom = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-30, 0, -10), max_bound=(10, 20, 10))
# # geom = geom.crop_point_cloud(pcd)
# # crop = vol.crop_point_cloud(pcd)
# # demo_crop_data = o3d.data.DemoCropPointCloud()
# # pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
# # vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
# # chair = vol.crop_point_cloud(pcd)
# oriented_bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(pcd.points)
# point_cloud_crop = pcd.crop(oriented_bounding_box)
# downpcd = point_cloud_crop.voxel_down_sample(voxel_size=0.5)
# o3d.io.write_point_cloud("pointcloud.pcd", downpcd)
# # View original point cloud with the cuboid, all 5 points present
# # o3d.visualization.draw_geometries([pcd, oriented_bounding_box])

# # View cropped point cloud with the cuboid, only 3 points present
# o3d.visualization.draw_geometries([point_cloud_crop, oriented_bounding_box])
# # o3d.visualization.draw_geometries([geom],
# #                                   zoom=0.7,
# #                                   front=[0.5439, -0.2333, -0.8060],
# #                                   lookat=[2.4615, 2.1331, 1.338],
# #                                   up=[-0.1781, -0.9708, 0.1608])
# # pcd.scale(10, center=[2.4615, 2.1331, 1.338])
# # downpcd = pcd.voxel_down_sample(voxel_size=0.5)
# # o3d.visualization.draw_geometries([pcd],
# #                                   zoom=10,
# #                                   front=[0.4257, -0.2125, -0.8795],
# #                                   lookat=[2.6172, 2.0475, 1.532],
# #                                   up=[-0.0694, -0.9768, 0.2024])
# # # pcd.transform(point_cloud)
# # print(pcd)

# # print("Paint pointcloud")
# # downpcd.paint_uniform_color([0, 1, 0])
# # o3d.visualization.draw([pcd])
# # o3d.visualization.draw([downpcd])
                                  
# # o3d.visualization.draw([pcd])
# # o3d.show()

# # pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
# # vis = o3d.visualization.Visualizer()
# # vis.create_window()
# # vis.add_geometry(pcd)
# # # o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.1)
# # vis.run()
# # o3d.visualization.draw([crop])
# # o3d.visualization.draw_geometries([pcd],
# #                                   zoom=0.7,
# #                                   front=[0.5439, -0.2333, -0.8060],
# #                                   lookat=[2.4615, 2.1331, 1.338],
# #                                   up=[-0.1781, -0.9708, 0.1608])


# # vis.show()

In [ ]:
t_abs_all = np.array(t_abs_all)
if KITTI == True:
    t_abs_gt = np.array(t_abs_gt)
# plt.plot(t_abs_all[:,2], t_abs_all[:,0])
# plt.show()
%matplotlib widget
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
# ax2 = fig.add_subplot(projection='3d')
# pt = (t_abs_all[0, 0], t_abs_all[0, 1], t_abs_all[0, 2])
ax.scatter(t_abs_all[:, 0], t_abs_all[:, 2], t_abs_all[:, 1], color=[(0, 1, 0)])
if point_cloud is not None:
    # print(point_cloud.shape)
    # point_cloud[point_cloud < 0] = -100
    
    # print(point_cloud)
    
    # point_cloud = np.array(random.sample(point_cloud,1000)).reshape(-1, 3)

    r = 0
    # for point in point_cloud:
    #     if point[2] > 0 and r % 1 == 0:
    # ax.scatter(point_cloud[:, 0], point_cloud[:, 2], point_cloud[:, 1], color=[(0, 0, 1)], s=0.0125)
    import open3d as o3d

    pcd = o3d.geometry.PointCloud()
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    ret = np.array(pcd.cluster_dbscan(eps=0.5, min_points=3))

    # print(ret.shape)
    print(point_cloud.shape)
    
    print(point_cloud[ret > 0].shape)
    print(t_abs_all.shape)
    # all_points = np.concatenate([point_cloud[ret > 0], t_abs_all, t_abs_gt])
    all_points = np.concatenate([point_cloud[ret > -1], t_abs_all])
    
    pcd.points = o3d.utility.Vector3dVector(all_points)

    red = [255, 0, 0]
    green = [0, 255, 0]
    blue = [0, 0, 255]
    purple = [255, 0, 255]
    orange = [255, 255, 0]
    color_palette = [red, orange, blue, purple]
    cluster_colors = {}

    for n in set(ret):
        cluster_colors[n] = color_palette[np.random.randint(len(color_palette))]

    colors = np.ndarray(all_points.shape)

    for i, color in enumerate(colors):
        colors[i] = cluster_colors[ret[i]]
    # colors[:point_cloud[ret>-1].shape[0]] = red
    colors[point_cloud[ret>-1].shape[0]:point_cloud[ret>-1].shape[0]+t_abs_all.shape[0]] = green
    # colors[point_cloud[ret>0].shape[0]+t_abs_all.shape[0]:] = blue
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # downpcd = pcd.voxel_down_sample(voxel_size=0.5)
    # ret = np.asarray(point_cloud).flatten()[ret]

    o3d.io.write_point_cloud("pointcloud_clustered.pcd", pcd)
    # with open('pointcloud.txt', 'w') as f:
    #     for point in point_cloud:
    #         f.write(str(point[0]) + ',' + str(point[1]) + ',' + str(point[2]) + '\n')

    # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, 0.5)
    # mesh.compute_vertex_normals()
    
    # o3d.io.write_triangle_mesh("mesh.obj", mesh)

maximum = None
minimum = None

if KITTI == True:
    ax.scatter(t_abs_gt[:, 0], t_abs_gt[:, 2], t_abs_gt[:, 1], color=(1, 0, 0))
    minimum = np.array(np.concatenate([t_abs_all, t_abs_gt])).min()
    maximum = np.array(np.concatenate([t_abs_all, t_abs_gt])).max()
else:
    minimum = np.array(t_abs_all).min()
    maximum = np.array(t_abs_all).max()
    
# ax.set_zscale(lower=minimum, upper=maxiumum, value=)
ax.set_xlim([minimum, maximum])
ax.set_ylim([minimum, maximum])
ax.set_zlim([minimum, 100])
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')
# plt.plot(t_abs_all[:, 0], t_abs_all[:, 1], t_abs_all[:, 2])
plt.show()



In [ ]:
# from vedo import *
# import numpy as np

# # pts = np.random.randn(200, 3)

# # with color mapping
# vpts = Points(point_cloud).ps(10)
# p = vpts.points()
# p[0] = [5,5,4] # make this point stick out
# vpts.points(p)

# vpts.cmap('viridis', pts[:,2], name="mydata")
# vpts.print()
# show(vpts, axes=1).close()

# # with rgb direct coloring
# cols = np.random.rand(200, 3)*255
# cols[0] = [255,0,0] # make it red
# vpts.cellIndividualColors(cols)

# show(vpts, axes=1).close()